In [4]:
from __future__ import division
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
# import matplotlib as mpl
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Sample data

In [18]:
df = pd.read_csv('../SF_311_Data-Analysis/data/cases_sample.csv', header=0, index_col=3, parse_dates=[4, 5, 6])

In [19]:
df = df.drop(['Unnamed: 0', 'X.1', 'X'], axis=1)

In [20]:
print(df['Opened'].min())
print(df['Opened'].max())

2008-07-01 07:27:00
2016-01-27 16:51:21


In [21]:
df.shape

(4985, 20)

In [14]:
df.head()

,Opened,Closed,Updated,Status,Status.Notes,Responsible.Agency,Category,Request.Type,Request.Details,Address,Supervisor.District,Neighborhood,Point,Source,Media.URL,Lat,Long,Census.Block,Zip,Map.Url
CaseID,,,,,,,,,,,,,,,,,,,,
1757188,2012-12-04 15:21:00,2012-12-06 04:39:00,2012-12-06 04:39:00,Closed,NaN,DPW BSM Queue,Street Defects,Utility_Excavation,Utility_Excavation,"2820 GREENWICH ST, SAN FRANCISCO, CA, 94123",2,Marina,"(37.797838468, -122.446243791)",Voice In,NaN,37.797838,-122.446244,60750128003,94123,http://open.mapquestapi.com/staticmap/v4/getma...
1834805,2012-12-30 08:39:28,2013-01-15 07:15:14,2013-01-15 07:15:14,Closed,Case Resolved -,zzRPD Urban Forestry Queue - DO NOT USE,Rec and Park Requests,Park - UrbanForestry,Broken_Limb,"Park Presidio Blvd, SAN FRANCISCO, CA, 94118",1,Inner Richmond,"(37.780122585, -122.4722852)",Voice In,NaN,37.780123,-122.472285,60750476001,94118,http://open.mapquestapi.com/staticmap/v4/getma...
3016735,2013-11-01 16:41:00,2015-04-16 04:05:00,2015-04-16 04:05:00,Closed,Case Transferred - Per process - cases are clo...,311 Service Request Queue - Hold,311 External Request,Graffiti,Graffiti - USPS,Intersection of PACIFIC AVE and WEBSTER ST,2,Pacific Heights,"(37.7935575960028, -122.433088431645)",Voice In,NaN,37.793558,-122.433088,60750132001,94123,http://open.mapquestapi.com/staticmap/v4/getma...
2643949,2013-07-26 09:57:04,2013-08-02 12:01:03,2013-08-02 12:01:03,Closed,DPT Abandoned Vehicles- Gone on arrival - 8-2-...,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Car2door,White - Honda - NA - 6LWD681,"1735 17TH AVE, SAN FRANCISCO, CA, 94122",7,NaN,"(37.755495625, -122.474745239)",Voice In,NaN,37.755496,-122.474745,60750303021,94116,http://open.mapquestapi.com/staticmap/v4/getma...
3600300,2014-04-30 14:52:36,2014-04-30 15:49:58,2014-04-30 15:49:58,Closed,Case Completed - resolved: WASTE NOT FOUND,Recology_Abandoned,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"549 LOMBARD ST, SAN FRANCISCO, CA, 94133",3,North Beach,"(37.803081142313, -122.410557299769)",Voice In,NaN,37.803081,-122.410557,60750104002,94133,http://open.mapquestapi.com/staticmap/v4/getma...


In [22]:
geo = pd.read_csv('../SF_311_Data-Analysis/data/Analysis_Neighborhoods.csv', header=0)
geo = geo.rename(columns={'GEOID': 'Census.Block'}).head()

In [37]:
pd.read_csv?

In [43]:
agesex = pd.read_csv('../SF_311_Data-Analysis/data/ACS_14_5YR-AGESEX/ACS_14_5YR_B01002_with_ann.csv', header=1, skiprows=0)
agesex = agesex.rename(columns={'Id2': 'Census.Block'}).head()

In [44]:
agesex.head()

,Id,Census.Block,Geography,Estimate; Median age -- - Total:,Margin of Error; Median age -- - Total:,Estimate; Median age -- - Male,Margin of Error; Median age -- - Male,Estimate; Median age -- - Female,Margin of Error; Median age -- - Female
0,1500000US060750101001,60750101001,"Block Group 1, Census Tract 101, San Francisco...",33.6,1.8,33.3,3.2,33.8,2.8
1,1500000US060750101002,60750101002,"Block Group 2, Census Tract 101, San Francisco...",40.9,4.6,33.6,8.9,56.8,11.6
2,1500000US060750102001,60750102001,"Block Group 1, Census Tract 102, San Francisco...",47.3,4.0,47.7,2.7,43.2,14.1
3,1500000US060750102002,60750102002,"Block Group 2, Census Tract 102, San Francisco...",34.8,3.3,32.7,1.2,36.7,7.1
4,1500000US060750102003,60750102003,"Block Group 3, Census Tract 102, San Francisco...",46.9,6.4,46.7,6.9,49.4,16.3


In [59]:
agesex['Census.Block'].head().apply(type)

0    <class 'numpy.int64'>
1    <class 'numpy.int64'>
2    <class 'numpy.int64'>
3    <class 'numpy.int64'>
4    <class 'numpy.int64'>
Name: Census.Block, dtype: object

In [48]:
df = df.join(agesex[['Census.Block', 'Estimate; Median age -- - Female', 'Estimate; Median age -- - Male']],
        on=['Census.Block'], how='left', rsuffix='_acs')

In [49]:
df.head()

,Opened,Closed,Updated,Status,Status.Notes,Responsible.Agency,Category,Request.Type,Request.Details,Address,Supervisor.District,Neighborhood,Point,Source,Media.URL,Lat,Long,Census.Block,Zip,Map.Url,Census.Block_geo,Estimate; Median age -- - Female,Estimate; Median age -- - Male
CaseID,,,,,,,,,,,,,,,,,,,,,,,
1757188,2012-12-04 15:21:00,2012-12-06 04:39:00,2012-12-06 04:39:00,Closed,NaN,DPW BSM Queue,Street Defects,Utility_Excavation,Utility_Excavation,"2820 GREENWICH ST, SAN FRANCISCO, CA, 94123",2,Marina,"(37.797838468, -122.446243791)",Voice In,NaN,37.797838,-122.446244,60750128003,94123,http://open.mapquestapi.com/staticmap/v4/getma...,NaN,NaN,NaN
1834805,2012-12-30 08:39:28,2013-01-15 07:15:14,2013-01-15 07:15:14,Closed,Case Resolved -,zzRPD Urban Forestry Queue - DO NOT USE,Rec and Park Requests,Park - UrbanForestry,Broken_Limb,"Park Presidio Blvd, SAN FRANCISCO, CA, 94118",1,Inner Richmond,"(37.780122585, -122.4722852)",Voice In,NaN,37.780123,-122.472285,60750476001,94118,http://open.mapquestapi.com/staticmap/v4/getma...,NaN,NaN,NaN
3016735,2013-11-01 16:41:00,2015-04-16 04:05:00,2015-04-16 04:05:00,Closed,Case Transferred - Per process - cases are clo...,311 Service Request Queue - Hold,311 External Request,Graffiti,Graffiti - USPS,Intersection of PACIFIC AVE and WEBSTER ST,2,Pacific Heights,"(37.7935575960028, -122.433088431645)",Voice In,NaN,37.793558,-122.433088,60750132001,94123,http://open.mapquestapi.com/staticmap/v4/getma...,NaN,NaN,NaN
2643949,2013-07-26 09:57:04,2013-08-02 12:01:03,2013-08-02 12:01:03,Closed,DPT Abandoned Vehicles- Gone on arrival - 8-2-...,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Car2door,White - Honda - NA - 6LWD681,"1735 17TH AVE, SAN FRANCISCO, CA, 94122",7,NaN,"(37.755495625, -122.474745239)",Voice In,NaN,37.755496,-122.474745,60750303021,94116,http://open.mapquestapi.com/staticmap/v4/getma...,NaN,NaN,NaN
3600300,2014-04-30 14:52:36,2014-04-30 15:49:58,2014-04-30 15:49:58,Closed,Case Completed - resolved: WASTE NOT FOUND,Recology_Abandoned,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,"549 LOMBARD ST, SAN FRANCISCO, CA, 94133",3,North Beach,"(37.803081142313, -122.410557299769)",Voice In,NaN,37.803081,-122.410557,60750104002,94133,http://open.mapquestapi.com/staticmap/v4/getma...,NaN,NaN,NaN


In [52]:
df.shape

(4985, 23)

In [51]:
df['Census.Block_geo'].isnull().sum()

4985

In [8]:
df[(df['Opened'] > '2014-01-01') & (df['Opened'] < '2016-01-01')].shape

(1957, 23)